# Probabilistic Grammar Fuzzing

Let us give grammars even more power by assigning _probabilities_ to individual expansions.  This allows us to control how many of each element should be produced, and thus allows us to _target_ our generated tests towards specific functionality.  We also show how to learn such probabilities from given sample inputs, and specifically direct our tests towards input features that are uncommon in these samples.

**Prerequisites**

* You should have read the [chapter on grammars](Grammars.ipynb).
* Our implementation hooks into the grammar-based fuzzer introduced in ["Efficient Grammar Fuzzing"](GrammarFuzzer.ipynb)
* For learning probabilities from samples, we make use of [parsers](Parser.ipynb)

## The Law of Leading Digits

In all our examples so far, you may have noted that inputs generated by a program differ quite a bit from "natural" inputs as they occur in real life.  This is true even for innocuous elements such as numbers – yes, the numbers we have generated so far actually _differ_ from numbers in the real world.  This is because in real-life sets of numerical data, the _leading significant digit_ is likely to be small: Actually, on average, the leading digit `1` occurs more than _six times_ as often as the leading digit `8` or `9`.  It has been shown that this result applies to a wide variety of data sets, including electricity bills, street addresses, stock prices, house prices, population numbers, death rates, lengths of rivers, physical and mathematical constants (Wikipedia).

This law, first observed by Newcomb \cite{Newcomb1881), was formalized by Benford in \cite{Benford1938).  Let us take a look at the conditions that determine the first digit of a number.  We can easily compute the first digit by converting the number into a string and take the first character:

In [ ]:
def first_digit_via_string(x):
    return ord(repr(x)[0]) - ord('0')

In [ ]:
first_digit_via_string(2001)

To do this mathematically, though, we have to take the fractional part of their logarithm, or formally

$$
d = 10^{\{\log_{10}(x)\}}
$$

where $\{x\}$ is the fractional part of $x$ (i.e. $\{1.234\} = 0.234$).

In [ ]:
import math

In [ ]:
def first_digit_via_log(x):
    frac, whole = math.modf(math.log10(x))
    return int(10 ** frac)

In [ ]:
first_digit_via_log(2001)

Most sets of "naturally" occurring numbers should not have any bias in the fractional parts of their logarithms, and hence, the fractional part $\{\log_{10}(x)\}$ is typically uniformly distributed.  However, the fractional parts for the individual digits are _not_ evenly distributed.  

For a number to start with a digit $d$, the condition $d < 10^{\{\log_{10}(x)\}} < d + 1$ must hold.  To start with the digit 1, the fractional part $\{\log_{10}(x)\}$ must thus be in the range

In [ ]:
(math.log10(1), math.log10(2))

To start with the digit 2, though, it must be in the range

In [ ]:
(math.log10(2), math.log10(3))

which is much smaller.  Formally, the probability $P(d)$ for a leading digit $d$ (again, assuming uniformly distributed fractional parts) is known as Benford's law:
$$
P(d) = \log_{10}(d + 1) - \log_{10}(d)
$$
which gives us:

In [ ]:
def prob_leading_digit(d):
    return math.log10(d + 1) - math.log10(d)

In [ ]:
print([(d, prob_leading_digit(d)) for d in range(1, 10)])

and you can see that a leading 1 is indeed six times a probable than a leading 9.

Benford's law has a number of applications.  Most notably, it can be used to detect "non-natural" numbers, i.e. numbers that apparently were created randomly rather than coming from a "natural" source.  if you write a scientific paper and fake data by putting in random numbers (for instance, [using our grammar fuzzer](GrammarFuzzer.ipynb) on integers), you will likely violate Benford's law, and this can indeed be spotted.  On the other hand, how would we proceed if we _wanted_ to create numbers that adhere to Benson's law?  To this end, we need to be able to _encode_ probabilities such as the above in our grammar, such that we can ensure that a leading digit is indeed a `1` in 30% of all cases.

## Specifying Probabilities

The goal of this chapter is to assign _probabilities_ to individual expansions in the grammar, such that we can express that some expansion alternatives should be favored over others.  This is not only useful to generate "natural"-looking numbers, but even more so to _direct_ test generation towards a specific goal.  If you recently have changed some code in your program, you would probably like to generate inputs that exercise precisely this code.  By raising the probabilities on the input elements associated with the changed code, you will get more tests that exercise the changed code.

Our concept for expressing probabilities is to _annotate_ individual expansions with attributes such as probabilities.  To this end, we allow that an expansion cannot only be a string, but also a _pair_ of a string and a set of attributes, as in

```python
    "<expr>":
        [("<term> + <expr>", opts(prob=0.1)),
         ("<term> - <expr>", opts(prob=0.2)),
         "<term>"]
```

Here, the `opts()` function would allow us to express probabilities for choosing the individual expansions.  The addition would have a probability of 10%, the subtraction of 20%.  The remaining probability (in this case 70%) is equally distributed over the non-attributed expansions (in this case the single last one).

Our `opts()` helper function returns a mapping of its arguments to values:

In [ ]:
def opts(**kwargs):
    return kwargs

In [ ]:
opts(prob=0.50)

We can now use pairs with `opts()` to assign probabilities to our expression grammar:

In [ ]:
import fuzzingbook_utils

In [ ]:
from GrammarFuzzer import GrammarFuzzer, all_terminals

In [ ]:
from Grammars import is_valid_grammar, EXPR_GRAMMAR, START_SYMBOL, crange

In [ ]:
PROBABILISTIC_EXPR_GRAMMAR = {
    "<start>":
        ["<expr>"],

    "<expr>":
        [("<term> + <expr>", opts(prob=0.1)),
         ("<term> - <expr>", opts(prob=0.2)),
         "<term>"],

    "<term>":
        [("<factor> * <term>", opts(prob=0.1)),
         ("<factor> / <term>", opts(prob=0.1)),
         "<factor>"
         ],

    "<factor>":
        ["+<factor>", "-<factor>", "(<expr>)",
            "<leadinteger>", "<leadinteger>.<integer>"],

    "<leadinteger>":
        ["<leaddigit><integer>", "<leaddigit>"],

    # Benford's law: frequency distribution of leading digits
    "<leaddigit>":
        [("1", opts(prob=0.301)),
         ("2", opts(prob=0.176)),
         ("3", opts(prob=0.125)),
         ("4", opts(prob=0.097)),
         ("5", opts(prob=0.079)),
         ("6", opts(prob=0.067)),
         ("7", opts(prob=0.058)),
         ("8", opts(prob=0.051)),
         ("9", opts(prob=0.046)),
         ],

    # Remaining digits are equally distributed
    "<integer>":
        ["<digit><integer>", "<digit>"],

    "<digit>":
        ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"],
}

In [ ]:
assert is_valid_grammar(PROBABILISTIC_EXPR_GRAMMAR)

This is how the grammar expansions are represented internally:

In [ ]:
PROBABILISTIC_EXPR_GRAMMAR["<leaddigit>"]

However, we typically access the expansion string and the associated probability via designated helper functions, `exp_string()` and `exp_prob()`:

In [ ]:
def exp_string(expansion):
    """Return the string to be expanded"""
    if isinstance(expansion, str):
        return expansion
    return expansion[0]

In [ ]:
exp_string(PROBABILISTIC_EXPR_GRAMMAR["<leaddigit>"][0])

In [ ]:
def exp_prob(expansion):
    """Return the specified probability, or None if unspecified"""
    if isinstance(expansion, str):
        return None
    return expansion[1]['prob']

In [ ]:
exp_prob(PROBABILISTIC_EXPR_GRAMMAR["<leaddigit>"][0])

Fortunately, our existing fuzzers have been set up to work well with grammars annotated this way.  They simply ignore all annotations:

In [ ]:
f = GrammarFuzzer(PROBABILISTIC_EXPR_GRAMMAR)
f.fuzz()

In [ ]:
from GrammarCoverageFuzzer import GrammarCoverageFuzzer

In [ ]:
f = GrammarCoverageFuzzer(PROBABILISTIC_EXPR_GRAMMAR)
f.fuzz()

## Checking Probabilities

To access probabilities, the function `prob_distribution()` returns a mapping of all expansions in a rule to their respective probabilities, also including those expansions without specified probabilities.  While doing so, it also checks for inconsistencies.

In [ ]:
def exp_probabilities(expansions, nonterminal="<symbol>"):
    probabilities = [exp_prob(expansion) for expansion in expansions]
    prob_dist = prob_distribution(probabilities, nonterminal)
    
    prob_mapping = {}
    for i in range(len(expansions)):
        expansion = exp_string(expansions[i])
        prob_mapping[expansion] = prob_dist[i]
    
    return prob_mapping

In [ ]:
def prob_distribution(probabilities, nonterminal="<symbol>"):
    epsilon = 0.00001

    number_of_unspecified_probabilities = probabilities.count(None)
    if number_of_unspecified_probabilities == 0:
        assert abs(sum(probabilities) - 1.0) < epsilon, \
            nonterminal + ": sum of probabilities must be 1.0"
        return probabilities

    sum_of_specified_probabilities = 0.0
    for p in probabilities:
        if p is not None:
            sum_of_specified_probabilities += p
    assert 0 <= sum_of_specified_probabilities <= 1.0, \
        nonterminal + ": sum of specified probabilities must be between 0.0 and 1.0"

    default_probability = ((1.0 - sum_of_specified_probabilities) / 
         number_of_unspecified_probabilities)
    all_probabilities = []
    for p in probabilities:
        if p is None:
            p = default_probability
        all_probabilities.append(p)

    assert abs(sum(all_probabilities) - 1.0) < epsilon
    return all_probabilities

Here's the mapping `exp_probabilities()` returns for the annotated `<leaddigit>` element:

In [ ]:
print(exp_probabilities(PROBABILISTIC_EXPR_GRAMMAR["<leaddigit>"]))

If no expansion is annotated, all expansions have the same likelihood of being selected, as in our previous grammar fuzzers.

In [ ]:
print(exp_probabilities(PROBABILISTIC_EXPR_GRAMMAR["<digit>"]))

`exp_probabilities()` distributes any remaining percentage across non-annotated expansions:

In [ ]:
exp_probabilities(PROBABILISTIC_EXPR_GRAMMAR["<expr>"])

We can use the checking capabilities of `exp_probabilities()` to check a probabilistic grammar for consistency:

In [ ]:
def is_valid_probabilistic_grammar(grammar, start_symbol=START_SYMBOL):
    if not is_valid_grammar(grammar, start_symbol):
        return False
   
    for nonterminal in grammar:
        expansions = grammar[nonterminal]
        prob_dist = exp_probabilities(expansions, nonterminal)
    
    return True

In [ ]:
assert is_valid_probabilistic_grammar(PROBABILISTIC_EXPR_GRAMMAR)

In [ ]:
assert is_valid_probabilistic_grammar(EXPR_GRAMMAR)

In [ ]:
from ExpectError import ExpectError

In [ ]:
with ExpectError():
    assert not is_valid_probabilistic_grammar({"<start>": [("1", opts(prob=0.5))]})

In [ ]:
with ExpectError():
    assert not is_valid_probabilistic_grammar({"<start>": [("1", opts(prob=1.5)), "2"]})

## Selecting by Probability

Now that we have seen how to specify probabilities for a grammar, we can actually implement probabilistic selection.  In our `ProbabilisticGrammarFuzzer`, it suffices to overload one method, namely `choose_node_expansion()`.  For each of the children we can choose from (typically all expansions of a symbol), we determine their probability (using `exp_probabilities()`), and make a weighted choice using `random.choices()` with a `weight` argument.

In [ ]:
import random

In [ ]:
class ProbabilisticGrammarFuzzer(GrammarFuzzer):
    def choose_node_expansion(self, node, possible_children):
        (symbol, tree) = node
        expansions = self.grammar[symbol]
        probabilities = exp_probabilities(expansions)

        weights = []
        for child in possible_children:
            child_weight = probabilities[all_terminals((node, child))]
            weights.append(child_weight)
            
        return random.choices(range(len(possible_children)), weights=weights)[0]

Our probabilistic grammar fuzzer works just like the non-probabilistic grammar fuzzer, except that it actually respects probability annotations.  Let us generate a couple of "natural" numbers that respect Benford's law:

In [ ]:
natural_fuzzer = ProbabilisticGrammarFuzzer(PROBABILISTIC_EXPR_GRAMMAR, start_symbol="<leadinteger>")
print([natural_fuzzer.fuzz() for i in range(20)])

In contrast, these numbers are pure random:

In [ ]:
integer_fuzzer = GrammarFuzzer(PROBABILISTIC_EXPR_GRAMMAR, start_symbol="<leadinteger>")
print([integer_fuzzer.fuzz() for i in range(20)])

Are the "natural" numbers really more "natural" than the random ones?  To show that `ProbabilisticGrammarFuzzer` indeed respects  the probabilistic annotations, let us create a specific fuzzer for the lead digit:

In [ ]:
leaddigit_fuzzer = ProbabilisticGrammarFuzzer(PROBABILISTIC_EXPR_GRAMMAR, start_symbol="<leaddigit>")
leaddigit_fuzzer.fuzz()

If we generate thousands of lead digits, their distribution should again follow Benford's law:

In [ ]:
trials = 10000

count = {}
for c in crange('0', '9'):
    count[c] = 0

for i in range(trials):
    count[leaddigit_fuzzer.fuzz()] += 1

print([(digit, count[digit] / trials) for digit in count])

Quod erat demonstrandum! The distribution is pretty much exactly as originally specified.  We now have a fuzzer where we can exercise control by specifying probabilities.

## Directed Fuzzing

By choosing probabilities wisely, we can direct fuzzing towards specific functions.  (Say, `urlparse()`),

## Learning Probabilities from Samples

In all our examples so far, you may have noted that inputs generated by a program differ quite a bit from "natural" inputs as they occur in real life.  This can be seen as a feature of test generation: The more generated inputs differ from natural inputs, the higher their chance of triggering new bugs – assuming they get accepted by the program under test, of course.  On the other hand, sometimes we also may want to generate inputs that share properties with "natural" inputs – for instance, if it comes to convincing developers that their code crashes not only for a pathological case, but also for inputs that may occur at any time.

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

Our exposition of Benford's law follows [this article](https://brilliant.org/wiki/benfords-law/).

## Exercises

Close the chapter with a few exercises such that people have things to do.  In Jupyter Notebook, use the `exercise2` nbextension to add solutions that can be interactively viewed or hidden:

* Mark the _last_ cell of the exercise (this should be a _text_ cell) as well as _all_ cells of the solution.  (Use the `rubberband` nbextension and use Shift+Drag to mark multiple cells.)
* Click on the `solution` button at the top.

(Alternatively, just copy the exercise and solution cells below with their metadata.)

### Exercise 1

Create a class `ProbabilisticGrammarCoverageFuzzer` that extends `GrammarCoverageFuzzer` with probabilistic capabilities.  The idea is to first cover all uncovered expansions (like `GrammarCoverageFuzzer`) and once all expansions are covered, to proceed by probabilities (like `ProbabilisticGrammarFuzzer`).  To this end, define new instances of the `choose_covered_node_expansion()` and `choose_uncovered_node_expansion()` methods that choose an expansion based on the given weights.  If you are an advanced programmer, realize the class via _multiple inheritance_ from `GrammarCoverageFuzzer` and `ProbabilisticGrammarFuzzer` to achieve this.

**Solution**.  With multiple inheritance, this is fairly easy; we just need to point the three methods to the right places:

In [ ]:
class ProbabilisticGrammarCoverageFuzzer(GrammarCoverageFuzzer, ProbabilisticGrammarFuzzer):
    # Choose uncovered expansions first
    def choose_node_expansion(self, node, possible_children):
        return GrammarCoverageFuzzer.choose_node_expansion(self, node, possible_children)

    # Among uncovered expansions, pick by (relative) probability
    def choose_uncovered_node_expansion(self, node, possible_children):
        return ProbabilisticGrammarFuzzer.choose_node_expansion(self, node, possible_children)
    
    # For covered nodes, pick by probability, too
    def choose_covered_node_expansion(self, node, possible_children):
        return ProbabilisticGrammarFuzzer.choose_node_expansion(self, node, possible_children)

In the first nine invocations, our fuzzer covers one digit after another:

In [ ]:
cov_leaddigit_fuzzer = ProbabilisticGrammarCoverageFuzzer(PROBABILISTIC_EXPR_GRAMMAR, start_symbol="<leaddigit>")
print([cov_leaddigit_fuzzer.fuzz() for i in range(9)])

After these, we again proceed by probabilities:

In [ ]:
trials = 10000

count = {}
for c in crange('0', '9'):
    count[c] = 0

for i in range(trials):
    count[cov_leaddigit_fuzzer.fuzz()] += 1

print([(digit, count[digit] / trials) for digit in count])

### Exercise 2

Test: \cite{Holler2012}

_Solution for the exercise_